In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from flask import Flask,render_template,url_for,request,make_response
import pandas as pd 
import pickle
from joblib import dump,load
import spacy
import re
import string
import datetime
import dateutil.parser
import base64
from bs4 import BeautifulSoup

In [ ]:
!python -m spacy download en_core_web_md

2023-05-08 14:03:50.662491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:

import re
import pandas as pd
import spacy
from sklearn.cluster import KMeans

# Load tweets from CSV file
tweets_df = pd.read_csv("tweets.csv", usecols=["text"])
tweets = tweets_df['text']

# Load spaCy model and GloVe word vectors
nlp = spacy.load('en_core_web_md')
glove_vectors = nlp.vocab.vectors.data

# Define tweet text preprocessing function
def preprocess_tweet_text(tweet_text):
    # Remove URLs
    tweet_text = re.sub(r'http\S+', '', tweet_text)
    # Remove mentions
    tweet_text = re.sub(r'@\S+', '', tweet_text)
    # Remove hashtags
    tweet_text = re.sub(r'#\S+', '', tweet_text)
    # Remove punctuation
    tweet_text = re.sub(r'[^\w\s]', '', tweet_text)
    # Lowercase
    tweet_text = tweet_text.lower()
    # Remove extra whitespace
    tweet_text = re.sub(r'\s+', ' ', tweet_text)
    return tweet_text

# Define candidate event extraction function
def extract_candidate_events(tweet_text):
    # Candidate event extraction
    doc = nlp(tweet_text)
    candidate_events = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ == 'VERB' and token.dep_ == 'ROOT':
                event = token.lemma_
                if len(event) > 2 and event not in candidate_events:
                    candidate_events.append(event)
    return candidate_events

# Define function to convert candidate events to vectors using GloVe
def get_event_vectors(candidate_events):
    event_vectors = []
    for event in candidate_events:
        event_vector = nlp.vocab[event].vector
        event_vectors.append(event_vector)
    return event_vectors

# Preprocess tweet text and extract candidate events from tweets
all_candidate_events = []
for tweet in tweets:
    preprocessed_tweet = preprocess_tweet_text(tweet)
    candidate_events = extract_candidate_events(preprocessed_tweet)
    all_candidate_events.extend(candidate_events)

# Convert candidate events to vectors using GloVe
event_vectors = get_event_vectors(all_candidate_events)

# Event clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(event_vectors)

# Print cluster labels
labels = kmeans.labels_
for i in range(num_clusters):
    cluster = [all_candidate_events[j] for j in range(len(all_candidate_events)) if labels[j] == i]
    print(f"Cluster {i}: {cluster}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0: ['help', 'need', 'need', 'need', 'let', 'try', 'need', 'let', 'want', 'need', 'want', 'try', 'want', 'want', 'need', 'want', 'help', 'let', 'help', 'let', 'try', 'let', 'try', 'use', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'need', 'want', 'need', 'need', 'let', 'let', 'need', 'help', 'let', 'want', 'help', 'want', 'want', 'try', 'try', 'try', 'want', 'use', 'need', 'let', 'need', 'help', 'use', 'want', 'need', 'let', 'need', 'want', 'tend', 'let', 'want', 'help', 'let', 'need', 'use', 'try', 'need', 'need', 'need', 'need', 'let', 'use', 'use', 'want', 'need', 'need', 'need', 'help', 'let', 'want', 'need', 'want', 'let', 'let', 'want', 'let', 'use', 'want', 'allow', 'use', 'use', 'let', 'use', 'want', 'use', 'use', 'use', 'need', 'want', 'try', 'want', 'let', 'try', 'try', 'need', 'try', 'let', 'use', 'try', 'let', 'need', 'want', 'try', 'try', 'need', 'try'

In [ ]:
import re
import csv
def extract_events(tweet):
    topics = {
    'Sports': ['game', 'match', 'tournament', 'sports', 'football', 'basketball', 'baseball', 'soccer', 'hockey', 'tennis', 'golf'],
    'Party': ['party', 'celebration', 'birthday', 'anniversary', 'fiesta', 'get together'],
    'TV': ['show', 'series', 'episode', 'netflix', 'hbo', 'tv', 'streaming', 'cable'],
    'Politics': ['politics', 'election', 'government', 'policy', 'president', 'vote', 'campaign', 'debate', 'congress', 'senate'],
    'Celebrity': ['celebrity', 'star', 'actor', 'actress', 'singer', 'musician', 'artist', 'famous', 'hollywood'],
    'Music': ['music', 'concert', 'festival', 'tour', 'singer', 'band', 'album', 'performance'],
    'Movie': ['movie', 'film', 'premiere', 'screening', 'cinema', 'hollywood', 'oscars'],
    'Food': ['food', 'restaurant', 'cuisine', 'chef', 'cooking', 'recipe', 'dining', 'menu'],
    'Concert': ['concert', 'music', 'festival', 'performance', 'show', 'live', 'band', 'singer', 'tour'],
    'Performance': ['performance', 'show', 'theater', 'broadway', 'stage', 'play', 'opera'],
    'Fitness': ['fitness', 'gym', 'workout', 'exercise', 'training', 'yoga', 'running', 'sports'],
    'Interview': ['interview', 'conversation', 'talk', 'discussion', 'podcast', 'chat'],
    'ProductRelease': ['product release', 'launch', 'new product', 'unveil', 'announcement'],
    'Meeting': ['meeting', 'conference', 'summit', 'convention', 'event', 'gathering'],
    'Fashion': ['fashion', 'style', 'designer', 'runway', 'collection', 'clothing', 'accessories'],
    'Finance': ['finance', 'economy', 'stock', 'investment', 'bank', 'money'],
    'School': ['school', 'education', 'university', 'college', 'student', 'teacher', 'class'],
    'AlbumRelease': ['album release', 'new music', 'singer', 'band', 'release'],
    'Religion': ['religion', 'faith', 'church', 'worship', 'spiritual'],
    'Conflict': ['conflict', 'war', 'terrorism', 'violence', 'protest', 'demonstration'],
    'Prize': ['prize', 'award', 'trophy', 'medal', 'winner', 'competition'],
    'Legal': ['legal', 'law', 'court', 'lawsuit', 'judge', 'attorney'],
    'Death': ['death', 'obituary', 'funeral', 'mourning', 'loss'],
    'Sale': ['sale', 'discount', 'bargain', 'offer', 'promotion', 'deal'],
    'VideoGameRelease': ['game release', 'new game', 'video game', 'gaming'],
    'Graduation': ['graduation', 'commencement', 'degree', 'diploma', 'cap and gown'],
    'Racing': ['racing', 'motorsport', 'formula 1', 'nascar', 'indycar', 'drag race'],
    'Fundraiser/Drive': ['fundraiser', 'charity', 'donation', 'drive', 'benefit'],
    'Exhibit': ['exhibit', 'art exhibit', 'museum', 'gallery', 'display'],
    'Celebration': ['celebration', 'festivity', 'holiday', 'party', 'event'],
    'Books': ['book', 'literature', 'reading', 'author', 'novel'],
    'Film': ['film', 'movie', 'cinema', 'premiere', 'screening'],
    'Opening/Closing': ['opening', 'closing', 'grand opening', 'ribbon cutting'],
    'Wedding': ['wedding', 'marriage', 'bride', 'groom', 'ceremony'],
    'Holiday': ['holiday', 'vacation', 'trip', 'travel', 'destination'],
    'Medical': ['medical', 'health', 'doctor', 'hospital', 'patient'],
    'Wrestling': ['wrestling', 'pro wrestling', 'wwe', 'wrestler', 'match'],
    'OTHER': []
    }
    events = []
    for topic, keywords in topics.items():
        for keyword in keywords:
            if re.search(keyword, tweet, re.IGNORECASE):
                events.append(topic)
                break
    return events


# read tweets from csv file
with open('tweets.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader) # skip header row
    for row in reader:
        tweet = row[1] # assuming tweet is in second column
        events = extract_events(tweet)
        if events:
            print(row[3]) # print tweet ID
            print(tweet) # print full tweet
            print(events) # print events extracted from tweet
            print()


VAMPIRIS “An awesome vampire thriller with bite!” https://t.co/dxYgrAa6Cj #Vampiris #CorsairAdventures #thriller #va…
bioterrorism
['Conflict']

hachi-san88: Leon and Claire. They fight bioterrorism and have busy days. That’s why they value the time they spend tog…
bioterrorism
['Conflict']

To fight bioterrorism, sir.
bioterrorism
['Conflict']

hachi-san88: Leon and Claire. They fight bioterrorism and have busy days. That’s why they value the time they spend… https://t.co/hcjq3VatR2
bioterrorism
['Conflict']

Is this a threat? You plan on committing some bioterrorism by actively spreading a disease within a community, in a… https://t.co/bLDezQcemr
bioterrorism
['Conflict']

Either you’re an alarmist or planning on doing a little bioterrorism... 🤔
bioterrorism
['Conflict']

Nice. I called them out for supporting bioterrorism
bioterrorism
['Conflict']

Similarly, there are lots of different disasters including epidemics, natural disasters, wars, bioterrorism, and we… https://t.co/NRP53s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP/tweets(NLP)1.csv",encoding='latin-1')
data.head()

,Sentiment,id,Date and Time,Query,username,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
import pandas as pd
import spacy

# load spaCy model
nlp = spacy.load("en_core_web_sm")

# read in tweets from CSV file
df = pd.read_csv("/content/drive/MyDrive/NLP/tweets(NLP)1.csv",encoding='latin-1', usecols=["Date and Time", "Tweet"])

# define function to extract events from tweet
def extract_events(tweet):
    doc = nlp(tweet)
    events = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "EVENT"]
    return events

# extract events from tweets in CSV file
for _, row in df.iterrows():
    tweet = row["Tweet"]
    date = row["Date and Time"]
    events = extract_events(tweet)
    if events:
        for event, label in events:
            print(f"{date}: {tweet} -> {event} ({label})")

Mon Apr 06 23:07:09 PDT 2009: Wishes I was on the Spring Fling Tour with Dawn &amp; neecee Sigh  G'knight -> the Spring Fling Tour (EVENT)
Tue Apr 07 03:59:24 PDT 2009: huh? I thought Hells Kitchen was a Ramsay thing. Who's this nobody guy? Get Ramsay back.  -> Get Ramsay (EVENT)
Tue Apr 07 04:59:25 PDT 2009: Martin and Sam aren't going to be at Soccer Six this year  -> Soccer Six (EVENT)
Tue Apr 07 07:03:45 PDT 2009: I am so mad that I am missing University Day today because of my stupid internship  -> University Day (EVENT)
Tue Apr 07 07:59:01 PDT 2009: studying for our exams (done = bio, halfway = english &amp; math and gonna start with spanish and history) while everone else is in VACATION  -> english &amp (EVENT)
Tue Apr 07 08:45:05 PDT 2009: Let's Go D-Backs!! Opening Day was great-- too bad I had to watch it from work    instead of from the ballpark.  Game 2 tonite !! -> Opening Day (EVENT)
Fri Apr 17 20:32:35 PDT 2009: I have to go to work in the a.m. &amp; I CAN'T sleep!  -> t

KeyboardInterrupt: ignored